In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [3]:
ratings_data = pd.read_csv('./podaci/rating.csv', parse_dates=['timestamp'])

In [4]:
model = tf.keras.models.load_model('./modeli/model_1.keras')

In [17]:
# filtered_ratings_df = ratings_data[ratings_data['rating'] > 3]
# print(filtered_ratings_df.shape)
# rand_userIds = np.random.choice(filtered_ratings_df['userId'].unique(), 
#                                 size=int(len(filtered_ratings_df['userId'].unique())*0.3), 
#                                 replace=False)
# ratings = filtered_ratings_df.loc[filtered_ratings_df['userId'].isin(rand_userIds)]
# ratings.to_csv('./podaci/rating_2.csv', index=False)
ratings = pd.read_csv('./podaci/rating_2.csv', parse_dates=['timestamp'])

In [18]:
movies_data = pd.read_csv('./podaci/movie.csv')

In [21]:
user_id_to_predict = 11
all_movie_ids = ratings_data['movieId'].unique()
rated_movie_ids_by_user = ratings[ratings['userId'] == user_id_to_predict]['movieId'].unique()
movies_not_rated_by_user = set(all_movie_ids) - set(rated_movie_ids_by_user)
movies_not_rated_list = sorted(list(movies_not_rated_by_user))
movies_to_predict = np.random.choice(movies_not_rated_list, size=10, replace=False)
print(movies_to_predict)
movies_to_predict = [2178, 18, 973, 320, 5411]

[12616 13529 15728  8109  5787 12222 15223 12883 14868 10658]


In [22]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
ratings.loc[:, 'userId'] = user_encoder.fit_transform(ratings['userId'])
ratings.loc[:, 'movieId'] = movie_encoder.fit_transform(ratings['movieId'])
user_index = user_encoder.transform([user_id_to_predict])[0]
movie_indices = movie_encoder.transform(movies_to_predict)

In [23]:
predictions = model.predict([np.array([user_index] * len(movie_indices)), np.array(movie_indices)])
real_ratings = predictions.flatten() + 0.5

1/1 [==============================] - 0s 20ms/step


In [24]:
movie_ratings = list(zip(movies_to_predict, real_ratings))
sorted_predictions = sorted(movie_ratings, key=lambda x: x[1], reverse=True)
print(sorted_predictions)

[(973, 5.3323736), (2178, 5.2837534), (5411, 4.9749994), (320, 4.952567), (18, 4.9476132)]


In [25]:
print(f"Predviđene ocjene za korisnika {user_id_to_predict}:")
for movie_id, rating in sorted_predictions:
    movie_title = movies_data[movies_data['movieId'] == movie_id]['title'].values[0]
    print(f"Film ID {movie_id}: {movie_title}, Predviđena ocjena {int(rating)}")

Predviđene ocjene za korisnika 11:
Film ID 973: Meet John Doe (1941), Predviđena ocjena 5
Film ID 2178: Frenzy (1972), Predviđena ocjena 5
Film ID 5411: Summer Holiday (1963), Predviđena ocjena 4
Film ID 320: Suture (1993), Predviđena ocjena 4
Film ID 18: Four Rooms (1995), Predviđena ocjena 4


In [67]:
user_id_to_recommend = 12
movies_not_rated_by_user = ratings[~ratings['userId'].isin([user_id_to_recommend])]['movieId'].unique()
user_index = user_encoder.transform([user_id_to_recommend])[0]
movie_indices_to_recommend = movie_encoder.transform(movies_not_rated_by_user)

In [68]:
predictions = model.predict([np.array([user_index] * len(movie_indices_to_recommend)), np.array(movie_indices_to_recommend)])
real_ratings = predictions.flatten() + 0.5
movie_ratings = list(zip(movies_not_rated_by_user, real_ratings))
sorted_recommendations = sorted(movie_ratings, key=lambda x: x[1], reverse=True)
print(sorted_recommendations[:10])

582/582 [==============================] - 1s 982us/step
[(1154, 5.42608), (255, 5.401637), (3659, 5.3553987), (2210, 5.352063), (6256, 5.3461165), (1356, 5.32868), (13258, 5.3156824), (12497, 5.311924), (13857, 5.3088274), (313, 5.3058167)]


In [71]:
top_n_movies = 0
print(f"Preporuke za korisnika ID {user_id_to_recommend}:")
for movie_id, rating in sorted_recommendations:  # Prikazujemo prvih 10 preporuka
    if movie_id in movies_data['movieId'].values:
      movie_title = movies_data[movies_data['movieId'] == movie_id]['title'].values[0]
      print(f"Naziv filma (ID {movie_id}): {movie_title}")
      top_n_movies += 1
      if top_n_movies == 10:
        break

Preporuke za korisnika ID 12:
Naziv filma (ID 1154): T-Men (1947)
Naziv filma (ID 255): Jerky Boys, The (1995)
Naziv filma (ID 3659): Quatermass 2 (Enemy from Space) (1957)
Naziv filma (ID 2210): Sabotage (1936)
Naziv filma (ID 6256): House with Laughing Windows, The (Casa dalle finestre che ridono, La) (1976)
Naziv filma (ID 1356): Star Trek: First Contact (1996)
Naziv filma (ID 313): Swan Princess, The (1994)
Naziv filma (ID 1093): Doors, The (1991)
Naziv filma (ID 6803): Phenomena (a.k.a. Creepers) (1985)
Naziv filma (ID 2637): Mummy's Hand, The (1940)
